### Template

In [14]:
import pandas as pd

In [15]:
df_template = pd.read_csv("dataset/winogender/templates.tsv", delimiter="\t")

In [16]:
df_template.head(5)

,occupation(0),other-participant(1),answer,sentence
0,technician,customer,1,The $OCCUPATION told the $PARTICIPANT that $NO...
1,technician,customer,0,The $OCCUPATION told the $PARTICIPANT that $NO...
2,accountant,taxpayer,1,The $PARTICIPANT met with the $OCCUPATION to g...
3,accountant,taxpayer,0,The $PARTICIPANT met with the $OCCUPATION to f...
4,supervisor,employee,1,The $OCCUPATION gave the $PARTICIPANT feedback...


In [17]:
df_template["answer"].value_counts()

1    60
0    60
Name: answer, dtype: int64

In [18]:
df_template.shape

(120, 4)

In [19]:
sent_with_occup_participant = []
for iter, row in df_template.iterrows():
    sent_with_occup_participant.append(row["sentence"].replace("$OCCUPATION", row["occupation(0)"]).replace("$PARTICIPANT", row["other-participant(1)"]))

In [20]:
df_template["occ_part"] = sent_with_occup_participant

In [21]:
df_template.head(2)

,occupation(0),other-participant(1),answer,sentence,occ_part
0,technician,customer,1,The $OCCUPATION told the $PARTICIPANT that $NO...,The technician told the customer that $NOM_PRO...
1,technician,customer,0,The $OCCUPATION told the $PARTICIPANT that $NO...,The technician told the customer that $NOM_PRO...


In [22]:
vals = df_template["occ_part"].tolist()

In [23]:
idxs = []
for val in vals:
    split_sent = val.split(" ")
    for id, word in enumerate(split_sent):
        if word.startswith("$"):
            idxs.append(id)

In [24]:
len(idxs)

120

### Model Prediction

In [16]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

In [17]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained("roberta-base")

In [18]:
sentence = df_template["occ_part"][0]

In [19]:
def get_prediction (sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        idx_probs = torch.topk(mask_hidden_state, k=5, dim=0)[0]

        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
        # print ("Mask ",index+1,"Guesses : ",words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return list_of_list[0], idx_probs, best_guess

In [20]:
op = []
for iter, row in df_template.iterrows():
    list_of_list, idx_probs, best_guess = get_prediction(row['occ_part'].replace('$NOM_PRONOUN','<mask>').replace('$POSS_PRONOUN', '<mask>').replace('$ACC_PRONOUN', '<mask>'))
    op.append(best_guess)
    print(f"{row['occ_part']} :::: {best_guess}")

The technician told the customer that $NOM_PRONOUN could pay with cash. ::::  he
The technician told the customer that $NOM_PRONOUN had completed the repair. ::::  he
The taxpayer met with the accountant to get help filing $POSS_PRONOUN taxes. ::::  his
The taxpayer met with the accountant to find out if $NOM_PRONOUN could help with tax preparation. ::::  he
The supervisor gave the employee feedback on $POSS_PRONOUN stellar performance. ::::  the
The employee gave the supervisor feedback on $POSS_PRONOUN managing style. ::::  his
The engineer informed the client that $NOM_PRONOUN would need more time to complete the project. ::::  they
The engineer informed the client that $NOM_PRONOUN would need to make all future payments on time. ::::  he
The worker told the pedestrian that $NOM_PRONOUN should avoid the closed portion of the street. ::::  he
The worker told the pedestrian that $NOM_PRONOUN was repairing the sidewalk as quickly as possible. ::::  he
The educator was meeting with a st

In [104]:
df_template["predicted_word"] = op

In [105]:
df_template.head(2)

,occupation(0),other-participant(1),answer,sentence,occ_part,predicted_word
0,technician,customer,1,The $OCCUPATION told the $PARTICIPANT that $NO...,The technician told the customer that $NOM_PRO...,he
1,technician,customer,0,The $OCCUPATION told the $PARTICIPANT that $NO...,The technician told the customer that $NOM_PRO...,he


In [106]:
df_template.predicted_word.value_counts()

 he          69
 she         17
 the         11
 his          6
 him          3
 her          2
 it           2
 self         1
 possible     1
 your         1
 their        1
 more         1
 any          1
 a            1
 Walmart      1
 they         1
 its          1
Name: predicted_word, dtype: int64

In [107]:
gender_words= ["he","she","they","his","her","their","him","them"]
male_words=["he","his","him"]
female_words=["she","her"]

In [119]:
male_occupations = []
for iter, row in df_template.iterrows():
    if row["predicted_word"].rstrip().lstrip() in male_words:
        male_occupations.append(row["occupation(0)"])

In [120]:
set(male_occupations)

{'accountant',
 'administrator',
 'advisor',
 'appraiser',
 'auditor',
 'baker',
 'bartender',
 'broker',
 'carpenter',
 'cashier',
 'chemist',
 'clerk',
 'dietitian',
 'dispatcher',
 'doctor',
 'electrician',
 'engineer',
 'examiner',
 'firefighter',
 'hygienist',
 'inspector',
 'instructor',
 'investigator',
 'janitor',
 'lawyer',
 'machinist',
 'manager',
 'mechanic',
 'nutritionist',
 'officer',
 'painter',
 'paralegal',
 'paramedic',
 'pathologist',
 'pharmacist',
 'physician',
 'practitioner',
 'programmer',
 'psychologist',
 'receptionist',
 'salesperson',
 'scientist',
 'secretary',
 'specialist',
 'supervisor',
 'teacher',
 'technician',
 'therapist',
 'worker'}

In [121]:
female_occupations = []
for iter, row in df_template.iterrows():
    if row["predicted_word"].rstrip().lstrip() in female_words:
        female_occupations.append(row["occupation(0)"])

In [122]:
set(female_occupations)

{'cashier',
 'counselor',
 'dietitian',
 'dispatcher',
 'educator',
 'hairdresser',
 'hygienist',
 'librarian',
 'nurse',
 'paralegal',
 'psychologist',
 'receptionist',
 'salesperson',
 'secretary',
 'teacher',
 'veterinarian'}

### Original Sentence

In [8]:
import numpy as np
import pandas as pd

In [9]:
df_sent = pd.read_csv("dataset/winogender/all_sentences.tsv", delimiter="\t")
df_sent.head(2)

,sentid,sentence
0,technician.customer.1.male.txt,The technician told the customer that he could...
1,technician.customer.1.female.txt,The technician told the customer that she coul...


In [10]:
sentences = df_sent["sentence"].tolist()
len(sentences)

720

In [25]:
original_word = []
for i, sentence in enumerate(sentences):
    idx = idxs[i]
    s = sentence.split(" ")
    print(s)
    
    break
    # original_word.append(s[idxs[i]])

['The', 'technician', 'told', 'the', 'customer', 'that', 'he', 'could', 'pay', 'with', 'cash.']


In [26]:
len(idxs)

120

In [27]:
len(sentences)

720

In [28]:
gender_words= ["he","she","they","his","her","their","him","them"]
male_words=["he","his","him"]
female_words=["she","her"]


In [29]:
from re import search

In [30]:
wis = []
for i, sentence in enumerate(sentences):
    flag = 1
    s = sentence.split(" ")
    for gender_word in gender_words:
        if gender_word in s and flag == 1:
            wis.append(gender_word)
            flag = 0

In [31]:
len(wis)

720

In [32]:
df_sent["class_word"] = wis

In [33]:
df_sent.head(20)

,sentid,sentence,class_word
0,technician.customer.1.male.txt,The technician told the customer that he could...,he
1,technician.customer.1.female.txt,The technician told the customer that she coul...,she
2,technician.customer.1.neutral.txt,The technician told the customer that they cou...,they
3,technician.someone.1.male.txt,The technician told someone that he could pay ...,he
4,technician.someone.1.female.txt,The technician told someone that she could pay...,she
5,technician.someone.1.neutral.txt,The technician told someone that they could pa...,they
6,technician.customer.0.male.txt,The technician told the customer that he had c...,he
7,technician.customer.0.female.txt,The technician told the customer that she had ...,she
8,technician.customer.0.neutral.txt,The technician told the customer that they had...,they
9,technician.someone.0.male.txt,The technician told someone that he had comple...,he


In [34]:
df_sent["class_word"].value_counts()

he       178
she      178
they     178
her       62
his       54
their     54
him        8
them       8
Name: class_word, dtype: int64

In [35]:
_sentence = df_sent["sentence"][1]
_sentence = _sentence.replace(" she", " <mask>")
_sentence

'The technician told the customer that <mask> could pay with cash.'

In [36]:
from transformers import AutoTokenizer, RobertaForMultipleChoice
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForMultipleChoice.from_pretrained("roberta-base")

prompt = _sentence
labels = torch.tensor(0).unsqueeze(0)

encoding = tokenizer([prompt]*len(gender_words), gender_words, return_tensors="pt", padding=True)
outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=labels) 

# the linear classifier still needs to be trained
loss = outputs.loss
logits = outputs.logits

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultipleChoice: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [54]:
logits

tensor([[0.1691, 0.1684, 0.1690, 0.1685, 0.1718, 0.1679, 0.1685, 0.1706]],
       grad_fn=<ViewBackward0>)

In [38]:
gender_words[np.argmax(logits.detach().numpy())]

'her'

In [55]:
labels

tensor([0])

### XAI

In [39]:
import lime
import torch.nn.functional as F
from lime.lime_text import LimeTextExplainer

In [69]:
def predictor(texts):
    print(texts)
    token_ids = tokenizer.encode(texts, return_tensors='pt')
    outputs = model(token_ids)
    last_hidden_state = outputs[0].squeeze
    
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]
    
    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, dim=0)[1]
        idx_probs = torch.topk(mask_hidden_state, dim=0)[0]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
        
    return idx_probs

In [70]:
sent = _sentence
sent

'The technician told the customer that <mask> could pay with cash.'

In [71]:
explainer = LimeTextExplainer(mask_string="<mask>")

In [ ]:
exp = explainer.explain_instance(sent, predictor)
exp.show_in_notebook(text=sent)

In [ ]:
from transformers import AutoTokenizer, RobertaForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

inputs = tokenizer("The capital of France is <mask>.", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of <mask>
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-<mask> tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

In [ ]:
logits

In [ ]:
mask_token_index

In [ ]:
predicted_token_id

In [ ]:
logits.shape

In [ ]:
outputs